# a Bernoulli MAB "solved" with Thompson Sampling
Thompson Sampling is a very interesting and _simple_ algorithm, for learning the best arm in a Multi-Arm-Bandit scenario. This notebook is intended as both a tutorial and a testbed for experimentation. The code simulating the bandit itself (the true world) is run from library python code, but all the rest is self-contained.

The inspiration is credited to https://peterroelants.github.io/posts/multi-armed-bandit-implementation/, which you are encouraged to follow before proceeding here.

## Imports and Google Colaboratory setup

In [ ]:
github_repo_name = 'bandits'
github_url = 'https://github.com/matanster/' + github_repo_name

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats
import random

if get_ipython().__class__.__module__ == "google.colab._shell":
    print('running in google colab')
    
    !git clone $github_url # clone the repo so that the python files of the repo are available to the runtime
    exec('import ' + github_repo_name + '.python_lib.bandit as bd')
    exec('from ' + github_repo_name + '.python_lib.util import unique_argmax')
    exec('from ' + github_repo_name + '.python_lib.plot import interval_binner, plot_series, plot_serie')


else:
    print('running locally')
    
    import python_lib.bandit as bd
    from python_lib.util import unique_argmax
    from python_lib.plot import interval_binner, plot_series, plot_serie
    
    #import ipycache # consult https://github.com/rossant/ipycache#usage for usage
    #%load_ext ipycache 
    #cache_file = 'cache-file' 

## The Algorithm and Our Simulation Code

The Thompson Sampling algorithm is very simple. It is essentially a Bayesian updating process with a small twist. We maintain and associate a beta distribution per arm of the Multi-Arm-Bandit and then iterate: 

At each round of interaction:

+ we sample each of these distributions, obtaining a sample value per arm
+ we then choose the arm that yielded the highest sample value 
+ we then choose that arm, and observe a reward given by it 
+ we update the distribution associated to that arm (review [here](https://www.youtube.com/watch?v=hKYvZF9wXkk) or elsewhere for the update rule we apply). 

You can see that this is a strictly a Bayesian process ― we continuously update our prior from the posterior, but only for one arm per round. At each round we choose the arm to play by the argmax of a random sample from each arm's prior distribution; we play that arm and update its associated distribution according to the reward we observed from it. This shall all be evident in the flow of the notebook that ensues. 

Here's our function for selecting the arm to choose:

In [ ]:
def thompson_select(beta_priors, bandits):   
    ''' samples from the prior distribution per bandit, and selects the argmax bandit '''

    beta_dist_samples = list(map(lambda k: np.random.beta(*beta_priors[k], size=1)[0], range(bandits.num_of_bandits)))
    selection = np.argmax(beta_dist_samples)
    
    if selection.size == 1:
        return selection.flat[0]
    else:
        return random.choice(selection) # tie break the argmax by random

And here's the complete algorithm running _i_ rounds of interaction:

In [ ]:
def simulate(bandits, beta_priors, n):
    ''' unleash online learning over n turns, keeping a record of every turn '''
    
    beta_priors = beta_priors.copy()
    
    round_records = []

    for i in range(n):

        round_record = dict(priors=list(beta_priors))

        k = thompson_select(beta_priors, bandits)
        outcome = bandits.pull(k)

        # the posterior computation here is specific to our simple case. 
        if outcome == 1:
            beta_priors[k] = (beta_priors[k][0]+1, beta_priors[k][1]) # increment the alpha param
        elif outcome == 0:
            beta_priors[k] = (beta_priors[k][0], beta_priors[k][1]+1) # increment the beta param
        else:
            raise ValueError()

        round_record.update(dict(k=k, outcome=outcome, posteriors=list(beta_priors)))

        round_records.append(round_record)
        
    return pd.DataFrame.from_records(round_records)

To evaluate success, we should look at multiple runs of the simulation, because the algorithm is probabilistic. So we also have a function here, that will run the simulation _n_ times for us:

In [ ]:
def simulate_n_times(bandits, beta_priors, turns, simulations):
    ''' run the same simulation m times, accumulating a record of every simulation's every turn '''
    simulation_dfs = []
    for i in range(simulations):
        simulation_df = simulate(bandits, beta_priors, turns)
        simulation_dfs.append(simulation_df)
    return simulation_dfs

These functions above will accumulate the data from all runs using pandas objects, for extra safe exploration of the results. In the same vein, the following functions will manipulate the accumulated experiment data, so that we can e.g. have the average of each turn _i_ across all of the _n_ simulations

In [ ]:
def compute_cross_simulation_stats(simulation_dfs, columns, stat_specs):
    ''' computes turn-wise stats across simulation dataframes.
        this is like building a data cube and aggregating the desired statistics on its new axis. '''
    
    # an intermediary dataframe where the index is the rounds, and each cell is the array 
    # of values across all simulations, for each original column
    rounds_df = pd.DataFrame(index=simulation_dfs[0].index, columns=columns)
    for column in columns:          
        for round in range(simulation_dfs[0].shape[0]):
            round_values = [simulation_df.iloc[round][column] for simulation_df in simulation_dfs]
            rounds_df[column].iloc[round] = round_values

    # the result dataframe where the index is the rounds, there is a column for every column+stat combo,
    # and the values are the implied statistics
    rounds_stats_df = pd.DataFrame(index=rounds_df.index, columns=[column + '-' + stat_spec['name'] for column in columns for stat_spec in stat_specs])
    for round in range(simulation_dfs[0].shape[0]):
        for column in columns:          
            for stat_spec in stat_specs:
                rounds_stats_df.iloc[round][column + '-' + stat_spec['name']] = \
                    stat_spec['fn'](rounds_df.iloc[round][column])
        
    return rounds_stats_df


def get_cross_simulation_stats(simulation_dfs):
    ''' computes the average and std per turn, for items of interest '''
    return compute_cross_simulation_stats(simulation_dfs, ['k', 'outcome'], [dict(name='avg', fn=np.mean), dict(name='std', fn=np.std)])

## Experiment Setup

### world setup
Let's now setup the real world which our algorithm should simulate against; that is simply the parameters of the bandits. In our imaginary world, each bandit is a Bernoulli bandit, having a single parameter which is the probability of that bandit emitting a success signal when pulled. Our simulation functions reach out for an implementation of a Bernoulli bandit to simulate this world.

In [ ]:
bandits = bd.BanditsVec(list(map(bd.Bernoulli, [0.03, 0.3, 0.4, 0.8, 0.9, 0.95])), bd.Bernoulli.best_bandit)


### priors initialization
We now initialize our beta priors to the uniform distribution. Lacking any outside information we have no reason to set them up any differently. Our notebook drives both the real world and the algorithm that runs against it. Obviously, it encodes the probabilities of the arms just one cell up, but we're after simulating how an agnostic algorithm would work...

In [ ]:
beta_priors = list(map(lambda bandit: (1,1), range(bandits.num_of_bandits)))

## Go algorithm go!

In [ ]:
# %%cache $cache_file simulation_dfs, round_stats
simulation_dfs = simulate_n_times(bandits, beta_priors, 200,1000)
round_stats = get_cross_simulation_stats(simulation_dfs)

### What did we get?
As stated, our experiment runs _i_ rounds of interaction, _n_ times over from scratch, because each run will have different results. The expectation and variance for each round across the simulations, will give us the concise picture of how the experiment went.

In [ ]:
plot_series('reward per turn over {} simulations of {} rounds each'.format(len(simulation_dfs), simulation_dfs[0].shape[0]),
            'turn', 'reward',            
            round_stats,
            [dict(column='outcome-avg', display_name='mean reward'), dict(column='outcome-std', display_name='std')], 
            lines=False,
            marker_size=3)

In [ ]:
plot_series('arm selection per turn over {} simulations of {} rounds each'.format(len(simulation_dfs), simulation_dfs[0].shape[0]),
            'turn', 'arm',            
            round_stats,
            [dict(column='k-avg', display_name='mean arm'), dict(column='k-std', display_name='std')], 
            lines=False,
            marker_size=3)

### What did our algorithm learn?
We look at the distributions of the means of the posterior parameters of each arm as they stand at the end of the simulation runs. Explicitly put, we take the final mean of the alpha and beta per arm over all simulations, and plot those beta distributions. Following some aggregation code, the plot follows.

In [ ]:
final_alpha_posteriors = pd.DataFrame(
    index=range(len(simulation_dfs)),
    columns = [arm for arm in range(bandits.num_of_bandits)])

final_beta_posteriors = pd.DataFrame(
    index=range(len(simulation_dfs)),
    columns = [arm for arm in range(bandits.num_of_bandits)])
    
for i, simulation_df in enumerate(simulation_dfs):
    for arm in range(bandits.num_of_bandits):
        final_alpha_posteriors.iloc[i][arm] = simulation_df.tail(1).iloc[0]['posteriors'][arm][0]
        final_beta_posteriors.iloc[i][arm]  = simulation_df.tail(1).iloc[0]['posteriors'][arm][1]
        
mean_alpha_posteriors = final_alpha_posteriors.mean()
mean_beta_posteriors  = final_beta_posteriors.mean()

In [ ]:
x = np.linspace(0,1,1000)
arm_estimates_df = pd.DataFrame(index=x)
for arm in range(bandits.num_of_bandits):   
    beta_distribution = scipy.stats.beta(mean_alpha_posteriors[arm], mean_beta_posteriors[arm])
    arm_estimates_df[arm] = [beta_distribution.pdf(i) for i in x]

In [ ]:
plot_series('final estimation of the probability density function per arm,<br>(averaged over all simulations)', '', '',
            arm_estimates_df, list(range(bandits.num_of_bandits)), 
            lines=True, fill=True, marker_size=1)

In [ ]:
params_df = pd.DataFrame(index = pd.Index(range(bandits.num_of_bandits), name='arm'))
params_df['true param'] = pd.Series(map(lambda bandit: bandit.success_prob, bandits.bandits))
params_df['mean(α)']    = pd.Series(mean_alpha_posteriors)
params_df['mean(β)']    = pd.Series(mean_beta_posteriors)
params_df['std(α)']     = pd.Series(final_alpha_posteriors.std())
params_df['std(β)']     = pd.Series(final_beta_posteriors.std())
display(params_df)

## TODO:

+ optimize the simulation functions by using numpy directly or otherwise
+ normalize the beta distributions into a probability distribution, at least for the plot (trivial)
+ fancy: make a dash page/app that chooses simulation runs dynamically from the simulations cube